# ADS 509 Team 1 Final Project

## Sentiment Analysis on Crypto Currencies

In [19]:
# import some necessary libraries

#!pip install psaw
import os
import datetime as dt
import re
import pandas as pd
import numpy as np

In [20]:
from psaw import PushshiftAPI
api = PushshiftAPI()

In [25]:
start_epoch=int(dt.datetime(2018, 1, 1).timestamp())
api_request_generator = api.search_submissions(after=start_epoch,
                                               subreddit='crypto',
                                              limit = 100)

In [26]:
post_df = pd.DataFrame([submission.d_ for submission in api_request_generator])

In [31]:
post_df.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_is_blocked',
       'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post',
       'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_richtext', 'link_flair_text_color', 'link_flair_type',
       'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts',
       'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'thumbnail', 'title', 'total_awards_rece

In [37]:
post_df['date'] = pd.to_datetime(post_df['created_utc'], utc=True, unit='s')
post_df['date']

0    2022-09-26 17:01:10+00:00
1    2022-09-26 15:33:41+00:00
2    2022-09-26 10:00:10+00:00
3    2022-09-26 01:37:59+00:00
4    2022-09-25 19:09:46+00:00
                ...           
95   2022-07-18 10:00:14+00:00
96   2022-07-18 10:00:10+00:00
97   2022-07-17 18:31:43+00:00
98   2022-07-16 20:17:28+00:00
99   2022-07-16 10:40:01+00:00
Name: date, Length: 100, dtype: datetime64[ns, UTC]

In [38]:
post_df[['author', 'date', 'title', 'selftext', 'url', 'subreddit', 'score', 'num_comments', 'num_crossposts', ]]

,author,date,title,selftext,url,subreddit,score,num_comments,num_crossposts
0,Zakepanka,2022-09-26 17:01:10+00:00,What is getting signed in XML Signature (XMLDs...,I want to parse an XML-DSIG file and it's cont...,https://www.reddit.com/r/crypto/comments/xop3e...,crypto,1,0,0
1,Sedulous_21,2022-09-26 15:33:41+00:00,Can someone tell me if my method of substituti...,Okay first thing first let me explain what I'm...,https://www.reddit.com/r/crypto/comments/xomng...,crypto,1,0,0
2,AutoModerator,2022-09-26 10:00:10+00:00,Weekly cryptography community and meta thread,Welcome to /r/crypto's weekly community thread...,https://www.reddit.com/r/crypto/comments/xoexr...,crypto,1,0,0
3,defkilla,2022-09-26 01:37:59+00:00,A couple of questions about libsodium...,I have read the documents for libsodium but I ...,https://www.reddit.com/r/crypto/comments/xo5fm...,crypto,1,0,0
4,yurichev,2022-09-25 19:09:46+00:00,SSL/TLS certificates survey (in wild) + RSA GC...,,https://yurichev.org/SSL_TLS_survey/,crypto,1,0,0
...,...,...,...,...,...,...,...,...,...
95,AutoModerator,2022-07-18 10:00:14+00:00,Monthly cryptography wishlist thread,This is another installment in a series of mon...,https://www.reddit.com/r/crypto/comments/w1vi5...,crypto,1,0,0
96,AutoModerator,2022-07-18 10:00:10+00:00,Weekly cryptography community and meta thread,Welcome to /r/crypto's weekly community thread...,https://www.reddit.com/r/crypto/comments/w1vhz...,crypto,1,0,0
97,greenappletree,2022-07-17 18:31:43+00:00,How difficult would it be for a modern tech to...,Hi supposed I created a series of custom ciphe...,https://www.reddit.com/r/crypto/comments/w1dz4...,crypto,1,0,0
98,HoldUpThereCowboy,2022-07-16 20:17:28+00:00,Is it possible to generate proofs of inclusion...,"I'm not sure if secret is the correct term, I'...",https://www.reddit.com/r/crypto/comments/w0pff...,crypto,1,0,0
